In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.pyplot import figure
from ipywidgets import interactive


# Data for the estimation of cone incidence vs leaf incidence
df_cone_incidence_leaf_incidence = pd.read_csv('../data/raw/economics/Leaf incidence_vs_Cone_incidence.csv', encoding='utf-8')

# Data preprocessing
df_cone_incidence_leaf_incidence = df_cone_incidence_leaf_incidence.dropna()

# Data for the estimation of yield loss vs cone incidence (change in quantity metric)
df_yield_loss_cone_incidence = pd.read_csv('../data/raw/economics/Cone incidence_vs_Cone color_and_Yield.csv', encoding='utf-8')
df_yield_loss_cone_incidence = df_yield_loss_cone_incidence[['Cone incidence', 'Yield Loss']]

# Data for the estimation of cone color vs cone incidence (change in quality metric)
df_cone_color_cone_incidence = pd.read_csv('../data/raw/economics/Cone incidence_vs_Cone color_and_Yield.csv', encoding='utf-8')
df_cone_color_cone_incidence = df_cone_color_cone_incidence[['Cone incidence', 'Cone Color']]


# Data
data_cone_incidence_leaf_incidence = df_cone_incidence_leaf_incidence.to_numpy()
data_yield_loss_cone_incidence = df_yield_loss_cone_incidence.to_numpy()
data_cone_color_cone_incidence = df_cone_color_cone_incidence.to_numpy()

# Linear regressions
a1, b1 = np.polyfit(data_cone_incidence_leaf_incidence[:, 0], data_cone_incidence_leaf_incidence[:, 1], 1)
a2, b2 = np.polyfit(data_yield_loss_cone_incidence[:, 0], data_yield_loss_cone_incidence[:, 1], 1)

# adjust y-values to make intercept 0
adjusted_y = data_cone_color_cone_incidence[:, 1] - 10

# fit a line with intercept of 0
a3, b3 = np.polyfit(data_cone_color_cone_incidence[:, 0], adjusted_y, 1)



# Load parameters
checkpoint1 = np.load('../reports/checkpoint_high_demand.npy', allow_pickle=True)
theta_high = checkpoint1[0]

checkpoint2 = np.load('../reports/checkpoint_moderate_demand.npy', allow_pickle=True)
theta_moderate = checkpoint2[0]

checkpoint3 = np.load('../reports/checkpoint_low_demand.npy', allow_pickle=True)
theta_low = checkpoint3[0]

# Define logistic sigmoid function
def sigmoid(z):

    g = 1 / (1 + np.exp(-z))
    
    return g

# QUALITY METRIC
# Function to estimate cone color given plant incidence
def cone_color(plant_incidence, n=10, D=1.19):
    
    # n is the number of leaves sampled and D is the index of dispersion
    n_D = n / D 
    
    leaf_incidence = leaf_incidence = 1 - (1 - plant_incidence) ** (1 / n_D)
    
    cone_incidence = a1 * leaf_incidence + b1
    # intercept fixed at 10 since that's the highest cone color rating
    cone_color = a3 * cone_incidence + 10
    
    return cone_color

# QUANTITY METRIC
# Function to estimate yield loss given plant incidence (probability of disease in plants)
def yield_loss(plant_incidence, n=10, D=1.19):
    
    # n is the number of leaves sampled and D is the index of dispersion
    n_D = n / D 
    
    leaf_incidence = 1 - (1 - plant_incidence) ** (1 / n_D)
    
    cone_incidence = a1 * leaf_incidence + b1
    
    yield_loss = a2 * cone_incidence
    
    return yield_loss


# Function to estimate percent revenue change given plant incidence
def revenue_change(plant_incidence, n=10, D=1.19, demand="high", variety="nugget"):
    
    if variety == "nugget":
        initial_yield = 2038.0
        initial_price = [2.5, 4.25, 6.0] #[low, mod, high] demand
        
    elif variety == "chinook":
        initial_yield = 1629.3333
        initial_price = [4.0, 5.0, 6.5]
        
    elif variety == "simcoe":
        initial_yield = 1752.0
        initial_price = [5.0, 6.5, 8.0]
        
    elif variety == "mosaic":
        initial_yield = 2134.0
        initial_price = [5.0, 6.5, 8.0]
        
    
    adjusted_yield = initial_yield * (1 - yield_loss(plant_incidence, n, D))
    
    if demand == "low":
        
        adjusted_price = initial_price[0] * sigmoid(theta_low[0] + theta_low[1] * cone_color(plant_incidence, n, D))
        adjusted_yield = initial_yield * (1 - yield_loss(plant_incidence, n, D))
        revenue = initial_price[0] * initial_yield
        adjusted_revenue = adjusted_price * adjusted_yield
        revenue_change = (adjusted_revenue - revenue) / revenue
    
    elif demand == "moderate":
        
        adjusted_price = initial_price[1] * sigmoid(theta_moderate[0] + theta_moderate[1] * cone_color(plant_incidence, n, D))
        revenue = initial_price[1] * initial_yield
        adjusted_revenue = adjusted_price * adjusted_yield
        revenue_change = (adjusted_revenue - revenue) / revenue
        
    elif demand == "high":
        
        adjusted_price = initial_price[2] * sigmoid(theta_high[0] + theta_high[1] * cone_color(plant_incidence, n, D))
        revenue = initial_price[2] * initial_yield
        adjusted_revenue = adjusted_price * adjusted_yield
        revenue_change = (adjusted_revenue - revenue) / revenue
        
    return revenue_change

## Results

In [2]:
def f(n=10, a=0):
   
    figure(figsize=(6, 4.5), dpi=100)
    x = np.linspace(0+a, 1, 1000)

    plt.plot(x, revenue_change(x, n=n, demand="high"), color='r', label='high demand')
    plt.plot(x, revenue_change(x, n=n, demand="moderate"), color='b', label='moderate demand')
    plt.plot(x, revenue_change(x, n=n, demand="low"), color='g', label='low demand')
    plt.ylim(-0.12, 0)
    plt.ylabel("percent revenue change")
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    plt.xlabel("probability of disease")
    plt.title("Revenue change given probability of disease")
    plt.legend()
    plt.show()
    
interactive_plot = interactive(f, n=(1, 100), a=(0, 0.999, 0.01))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(IntSlider(value=10, description='n', min=1), FloatSlider(value=0.0, description='a', max…